In [ ]:
# !pip install -r requirements.txt

In [ ]:
""" Imports"""

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pathlib
from functions import *
import shutil

from tensorflow.python.client import device_lib 
# print(device_lib.list_local_devices())

!nvidia-smi -L

# Load the TensorBoard notebook extension.
# %load_ext tensorboard

# multi GPU strategy
strategy = tf.distribute.MirroredStrategy() # you can define which gpus to use tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

# mixed precision policy
tf.keras.mixed_precision.set_global_policy('mixed_float16') # On TPUs, use 'mixed_bfloat16'

print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
data_dir = 'Dataset'
num_runs = 20

batch_size_per_replica = 16
img_height = 512
img_width = 512

buffer_size = 4000
batch_size = batch_size_per_replica * strategy.num_replicas_in_sync

model_choice = 0 #for custom model[0], Resnet50[1], ConvNext[2], EfficientNetv2[3]
filters = 16
epochs = 400

shutil.rmtree('Models', ignore_errors=True)

In [ ]:
# loop over all runs
for run in range(num_runs):
    
    seed = np.random.randint(0, 1000)# set seed
    print('Training Seed: ', seed)
    
    # preprocess data
    !python Preprocess_data.py
    # get data, split, augment, and save in `Dataset` folder
    
    Train_dir = 'Dataset/train'
    Test_dir = 'Dataset/test'
    
    Train_dir = pathlib.Path(Train_dir)
    Test_dir = pathlib.Path(Test_dir)
    
    # print number of images in each folder
    image_count = len(list(Train_dir.glob('*/*.jpg')))
    print('Number of images in Train folder: ', image_count)
    image_count = len(list(Test_dir.glob('*/*.jpg')))
    print('Number of images in Test folder: ', image_count)
    
    # load data
    train_ds, val_ds, test_ds, class_names = get_data_sets(Train_dir=Train_dir,
                                                           Test_dir=Test_dir,
                                                           Seed=seed,
                                                           Batch_size=batch_size,
                                                           Buffer=buffer_size,
                                                           IMG_HEIGHT=img_height,
                                                           IMG_WIDTH=img_width)
    
    print(class_names)

    # visualize some data
    plt.figure(figsize=(10, 10))
    for images, labels in train_ds.take(1):
        for i in range(6):
            ax = plt.subplot(3, 2, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(class_names[labels[i]])
            plt.axis("off")
            
    plt.show()

    num_classes = len(class_names)
    
    # load the model
    model, model_folder = my_model(choice=model_choice,
                                    img_height=img_height,
                                    img_width=img_width,
                                    strategy=strategy,
                                    num_classes=num_classes,
                                    filters=filters,
                                    run=run)
    
    # train the model
    model = train_net(model=model,
                      model_folder=model_folder,
                      train_ds=train_ds,
                      val_ds=val_ds,
                      test_ds=test_ds,
                      epochs=epochs,
                      strategy=strategy)

print('Done')